In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/gravitational_lens/dataset.zip" -d "/content/"


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/val/no/531.npy  
  inflating: /content/dataset/val/no/257.npy  
  inflating: /content/dataset/val/no/243.npy  
  inflating: /content/dataset/val/no/525.npy  
  inflating: /content/dataset/val/no/1099.npy  
  inflating: /content/dataset/val/no/1927.npy  
  inflating: /content/dataset/val/no/1933.npy  
  inflating: /content/dataset/val/no/519.npy  
  inflating: /content/dataset/val/no/1066.npy  
  inflating: /content/dataset/val/no/1700.npy  
  inflating: /content/dataset/val/no/294.npy  
  inflating: /content/dataset/val/no/2209.npy  
  inflating: /content/dataset/val/no/280.npy  
  inflating: /content/dataset/val/no/1714.npy  
  inflating: /content/dataset/val/no/1072.npy  
  inflating: /content/dataset/val/no/2235.npy  
  inflating: /content/dataset/val/no/1728.npy  
  inflating: /content/dataset/val/no/2221.npy  
  inflating: /content/dataset/val/no/733.npy  
  inflating: /content/dataset/val/no/727.npy 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

class LensingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir  # Path to dataset (train or validation)
        self.transform = transform
        self.classes = ["no", "sphere", "vort"]  # List of classes (substructure types)
        self.files = []  # List to store file paths and labels

        # Iterate over each class folder and store file paths with labels
        for label, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)  # e.g., "train/no"
            for file in os.listdir(class_path):  # Iterate over .npy files
                if file.endswith(".npy"):  # Check if it's a .npy file
                    self.files.append((os.path.join(class_path, file), label))  # Store (filepath, label)

    def __len__(self):
        return len(self.files)  # Returns total number of files

    def __getitem__(self, idx):
        file_path, label = self.files[idx]  # Get file path and label
        image = np.load(file_path)  # Load the .npy file (NumPy array)
        image = torch.tensor(image, dtype=torch.float32)  # Convert NumPy array to PyTorch tensor

        if self.transform:
            image = self.transform(image)  # Apply transformations if provided

        return image, label  # Return image tensor and label


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image
# Define dataset paths
train_dir = "/content/dataset/train"
val_dir = "/content/dataset/val"

transform = transforms.Compose([
    transforms.Lambda(lambda x: Image.fromarray((x.squeeze(0).numpy() * 255).astype('uint8'))),  # Convert tensor to PIL
    transforms.Resize((224, 224)),  # Resize for ResNet
    transforms.ToTensor(),  # Convert back to tensor
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert 1-channel to 3-channel
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize for 3 channels
])


# Load train and validation datasets
train_dataset = LensingDataset(train_dir, transform=transform)
val_dataset = LensingDataset(val_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Train samples: 30000
Validation samples: 7500


In [ ]:
#!ls -R /content/dataset/


In [ ]:
# Get one batch of images
data_iter = iter(train_loader)
images, labels = next(data_iter)

print("Batch shape:", images.shape)
print("Labels:", labels)


Batch shape: torch.Size([32, 3, 224, 224])
Labels: tensor([2, 0, 2, 1, 0, 0, 2, 2, 2, 1, 2, 1, 2, 0, 1, 2, 2, 1, 2, 1, 0, 2, 1, 1,
        1, 2, 2, 2, 0, 1, 0, 1])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
## Tried basic cnn model first but the model was not performing well (33%accuracy) that is classic case since there 3 class so it was just random guessing.Even with fine tuning it will not reach above (40%)

# # Import necessary libraries
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import torchvision.transforms as transforms
# import numpy as np
# import os

# # ============================
# # Define Custom Dataset Class
# # ============================
# class LensingDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.classes = ["no", "sphere", "vort"]
#         self.files = []

#         for label, class_name in enumerate(self.classes):
#             class_path = os.path.join(root_dir, class_name)
#             for file in os.listdir(class_path):
#                 if file.endswith(".npy"):
#                     self.files.append((os.path.join(class_path, file), label))

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, idx):
#         file_path, label = self.files[idx]
#         image = np.load(file_path)  # Load the .npy file
#         image = torch.tensor(image, dtype=torch.float32)  # Convert to tensor

#         if self.transform:
#             image = self.transform(image)

#         return image, label

# # ============================
# # Define CNN Model
# # ============================
# class LensCNN(nn.Module):
#     def __init__(self):
#         super(LensCNN, self).__init__()

#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

#         self.pool = nn.MaxPool2d(2, 2)  # Reduces feature map size

#         self.fc1 = nn.Linear(128 * 18 * 18, 256)  # Adjusted for 150x150 images
#         self.fc2 = nn.Linear(256, 3)  # 3 output classes

#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))

#         x = torch.flatten(x, start_dim=1)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)

#         return x

# # ============================
# # Define Transformations
# # ============================
# transform = transforms.Compose([
#     lambda x: torch.tensor(x, dtype=torch.float32).unsqueeze(0).squeeze(1)  # Remove extra dimension
# ])

# # ============================
# # Load Dataset and Create DataLoaders
# # ============================
# train_dir = "/content/dataset/train"
# val_dir = "/content/dataset/val"

# train_dataset = LensingDataset(train_dir, transform=transform)
# val_dataset = LensingDataset(val_dir, transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# # ============================
# # Set Up Model, Loss, and Optimizer
# # ============================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = LensCNN().to(device)  # Move model to GPU if available
# criterion = nn.CrossEntropyLoss()  # Multi-class classification loss
# optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Adam optimizer
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)  # Reduce LR on plateau

# # ============================
# # Training Loop
# # ============================
# def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10):
#     for epoch in range(epochs):
#         model.train()
#         running_loss, correct = 0.0, 0

#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)

#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
#             correct += (outputs.argmax(1) == labels).sum().item()

#         train_loss = running_loss / len(train_loader)
#         train_acc = correct / len(train_dataset)

#         # Validation step
#         model.eval()
#         val_loss, val_correct = 0.0, 0

#         with torch.no_grad():
#             for images, labels in val_loader:
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)

#                 val_loss += loss.item()
#                 val_correct += (outputs.argmax(1) == labels).sum().item()

#         val_loss /= len(val_loader)
#         val_acc = val_correct / len(val_dataset)

#         # Reduce LR if needed
#         scheduler.step(val_loss)

#         print(f"Epoch {epoch+1}/{epochs} - "
#               f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
#               f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# # ============================
# # Train the Model
# # ============================
# train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10)


In [ ]:
# Even though model is trained on only 10 epoch and only 4th layer is unfrezed the model performs extreamly well if given more epochs and unfreezed further layer it will perform even better
#test accuracy 96.26% and validation accuracy 93.05%
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np

# Load pretrained ResNet-18
model = resnet18(weights=ResNet18_Weights.DEFAULT)
num_features = model.fc.in_features

# Modify final layer and add dropout
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # Dropout to prevent overfitting
    nn.Linear(num_features, 3)
)

# Unfreeze last few layers
for param in list(model.parameters())[-10:]:
    param.requires_grad = True

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer with weight decay (L2 regularization)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.squeeze(1)),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Early stopping parameters
early_stopping_patience = 3
best_val_loss = np.inf
patience_counter = 0

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total
    scheduler.step()

    # Validation
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.4f}")

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter if validation loss improves
    else:
        patience_counter += 1

    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered!")
        break


Epoch 1/10 - Train Loss: 1.1307, Train Acc: 0.3309 | Val Loss: 1.1221, Val Acc: 0.3333
Epoch 2/10 - Train Loss: 1.0170, Train Acc: 0.4404 | Val Loss: 0.8281, Val Acc: 0.6169
Epoch 3/10 - Train Loss: 0.5385, Train Acc: 0.7784 | Val Loss: 0.3794, Val Acc: 0.8597
Epoch 4/10 - Train Loss: 0.3453, Train Acc: 0.8716 | Val Loss: 0.3273, Val Acc: 0.8736
Epoch 5/10 - Train Loss: 0.2768, Train Acc: 0.8992 | Val Loss: 0.2562, Val Acc: 0.9095
Epoch 6/10 - Train Loss: 0.1943, Train Acc: 0.9331 | Val Loss: 0.2240, Val Acc: 0.9189
Epoch 7/10 - Train Loss: 0.1624, Train Acc: 0.9427 | Val Loss: 0.2337, Val Acc: 0.9220
Epoch 8/10 - Train Loss: 0.1426, Train Acc: 0.9503 | Val Loss: 0.2160, Val Acc: 0.9241
Epoch 9/10 - Train Loss: 0.1202, Train Acc: 0.9578 | Val Loss: 0.2182, Val Acc: 0.9303
Epoch 10/10 - Train Loss: 0.1074, Train Acc: 0.9626 | Val Loss: 0.2072, Val Acc: 0.9305
